In [43]:
import pandas as pd


In [44]:
#Scrape the results dataom FBRef - using the 23/24 season

df = pd.read_html('https://fbref.com/en/comps/9/2023-2024/schedule/2023-2024-Premier-League-Scores-and-Fixtures', # allows us to read tables from html - paste in the link and css selectors
                  attrs={"id":"sched_2023-2024_9_1"})[0] # add the [0] to return the first item in the list - looks better

In [45]:
df.head()

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes
0,1.0,Fri,2023-08-11,20:00,Burnley,0.3,0–3,1.9,Manchester City,21572.0,Turf Moor,Craig Pawson,Match Report,NaN
1,1.0,Sat,2023-08-12,12:30,Arsenal,0.8,2–1,1.2,Nott'ham Forest,59984.0,Emirates Stadium,Michael Oliver,Match Report,NaN
2,1.0,Sat,2023-08-12,15:00,Everton,2.7,0–1,1.5,Fulham,39940.0,Goodison Park,Stuart Attwell,Match Report,NaN
3,1.0,Sat,2023-08-12,15:00,Sheffield Utd,0.5,0–1,1.9,Crystal Palace,31194.0,Bramall Lane,John Brooks,Match Report,NaN
4,1.0,Sat,2023-08-12,15:00,Brighton,4.0,4–1,1.5,Luton Town,31872.0,The American Express Community Stadium,David Coote,Match Report,NaN


In [102]:
# remove the 'nan' row from the above data - Fbref uses a 'nan' row as a separator, so we'll remove those rows
df = df.dropna(subset=['Home'])

In [103]:
df.head()

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes,home_goals,away_goals,result
0,1.0,Fri,2023-08-11,20:00,Burnley,0.3,0–3,1.9,Manchester City,21572.0,Turf Moor,Craig Pawson,Match Report,NaN,0.0,3.0,away_win
1,1.0,Sat,2023-08-12,12:30,Arsenal,0.8,2–1,1.2,Nott'ham Forest,59984.0,Emirates Stadium,Michael Oliver,Match Report,NaN,2.0,1.0,home_win
2,1.0,Sat,2023-08-12,15:00,Everton,2.7,0–1,1.5,Fulham,39940.0,Goodison Park,Stuart Attwell,Match Report,NaN,0.0,1.0,away_win
3,1.0,Sat,2023-08-12,15:00,Sheffield Utd,0.5,0–1,1.9,Crystal Palace,31194.0,Bramall Lane,John Brooks,Match Report,NaN,0.0,1.0,away_win
4,1.0,Sat,2023-08-12,15:00,Brighton,4.0,4–1,1.5,Luton Town,31872.0,The American Express Community Stadium,David Coote,Match Report,NaN,4.0,1.0,home_win


In [104]:
# Extract unique team names from column 'Home'
team_names = df['Home'].unique()

# Display the unique team names
print("Teams:")
for team in team_names:
    print(team)

Teams:
Burnley
Arsenal
Everton
Sheffield Utd
Brighton
Bournemouth
Newcastle Utd
Brentford
Chelsea
Manchester Utd
Nott'ham Forest
Fulham
Liverpool
Wolves
Tottenham
Manchester City
Aston Villa
West Ham
Crystal Palace
Luton Town


In [47]:
# Split the 'Score' column into 'Home Goals' and 'Away Goals' columns
df[['home_goals', 'away_goals']] = df['Score'].str.split('–', expand=True).astype(float)

In [48]:
# Determine the match outcome from the home and away goals

def get_outcome(row):
    if row['home_goals'] > row['away_goals']:
        return 'home_win'
    elif row['home_goals'] < row['away_goals']:
        return 'away_win'
    else:
        return 'draw'

# Create an outcome column following the above to record match winner
df['result'] = df.apply(get_outcome, axis=1)

In [49]:
# this tells us the columns we have in our DF

df.columns

Index(['Wk', 'Day', 'Date', 'Time', 'Home', 'xG', 'Score', 'xG.1', 'Away',
       'Attendance', 'Venue', 'Referee', 'Match Report', 'Notes', 'home_goals',
       'away_goals', 'result'],
      dtype='object')

In [51]:
# Calc number of fixtures
total_games = len(df)

# Calc home wins, away wins, and draws
home_wins = len(df[df['result'] == 'home_win'])
away_wins = len(df[df['result'] == 'away_win'])
draws = len(df[df['result'] == 'draw'])

# Calc win rates
home_win_rate = home_wins / total_games
away_win_rate = away_wins / total_games
draw_rate = draws / total_games

# Calc home-field advantage
home_field_advantage = home_win_rate - away_win_rate

print("Home Win Rate:", home_win_rate)
print("Away Win Rate:", away_win_rate)
print("Draw Rate:", draw_rate)
print("Home-Field Advantage:", home_field_advantage)

Home Win Rate: 0.41371158392434987
Away Win Rate: 0.2907801418439716
Draw Rate: 0.29550827423167847
Home-Field Advantage: 0.12293144208037826


In [52]:
# Create Dict to store data for each team
team_data = {}

# Loop through each team
for team in team_names:
    # Filter rows for the teams as Home Team
    home_team_rows = df[df['Home'] == team]

    # Calculate average goals for and against as Home Team
    avg_home_goals_for = home_team_rows['home_goals'].mean()
    avg_away_goals_against = home_team_rows['away_goals'].mean()

    # Filter rows for the specific team as Away Team
    away_team_rows = df[df['Away'] == team]

    # Calculate average goals for and against as Away Team
    avg_away_goals_for = away_team_rows['away_goals'].mean()
    avg_home_goals_against = away_team_rows['home_goals'].mean()

    # Calculate average goals for and against for each team
    OFF_rating = (avg_home_goals_for + avg_away_goals_for) / 2
    DEF_rating = (avg_home_goals_against + avg_away_goals_against) / 2

    # Store calculated data in the dictionary
    team_data[team] = {
        'Home Goals For': avg_home_goals_for,
        'Away Goals For': avg_away_goals_for,
        'Home Goals Against': avg_home_goals_against,
        'Away Goals Against': avg_away_goals_against,
        'OFF Rating': OFF_rating,
        'DEF Rating': DEF_rating
    }

# Display calculated data for each team
print("Team Data: AVG Goals & Ratings")
print()
# loop through each team to assign the team data
for team, data in team_data.items():
    print(f"{team}:")
    print(f"  Home Goals For: {data['Home Goals For']:.5f}")
    print(f"  Home Goals Against: {data['Home Goals Against']:.5f}")
    print(f"  Away Goals For: {data['Away Goals For']:.5f}")
    print(f"  Away Goals Against: {data['Away Goals Against']:.5f}")
    print(f"  OFF Rating: {data['OFF Rating']:.2f}")
    print(f"  DEF Rating: {data['DEF Rating']:.2f}")
    print()


Team Data: AVG Goals & Ratings

Burnley:
  Home Goals For: 1.00000
  Home Goals Against: 1.84211
  Away Goals For: 1.15789
  Away Goals Against: 2.26316
  OFF Rating: 1.08
  DEF Rating: 2.05

Arsenal:
  Home Goals For: 2.52632
  Home Goals Against: 0.68421
  Away Goals For: 2.26316
  Away Goals Against: 0.84211
  OFF Rating: 2.39
  DEF Rating: 0.76

Everton:
  Home Goals For: 1.15789
  Home Goals Against: 1.73684
  Away Goals For: 0.94737
  Away Goals Against: 0.94737
  OFF Rating: 1.05
  DEF Rating: 1.34

Sheffield Utd:
  Home Goals For: 1.00000
  Home Goals Against: 2.47368
  Away Goals For: 0.84211
  Away Goals Against: 3.00000
  OFF Rating: 0.92
  DEF Rating: 2.74

Brighton:
  Home Goals For: 1.57895
  Home Goals Against: 1.84211
  Away Goals For: 1.31579
  Away Goals Against: 1.42105
  OFF Rating: 1.45
  DEF Rating: 1.63

Bournemouth:
  Home Goals For: 1.42105
  Home Goals Against: 2.05263
  Away Goals For: 1.42105
  Away Goals Against: 1.47368
  OFF Rating: 1.42
  DEF Rating: 1.7

In [55]:
# Target just OFF & DEF for each team from the above

for team, stats in team_data.items():
    off_rating = stats.get('OFF Rating')
    def_rating = stats.get('DEF Rating')
    
    print(f"Team: {team}")
    print(f"OFF Rating: {off_rating}")
    print(f"DEF Rating: {def_rating}")
    print("-" * 30)

Team: Burnley
OFF Rating: 1.0789473684210527
DEF Rating: 2.052631578947368
------------------------------
Team: Arsenal
OFF Rating: 2.394736842105263
DEF Rating: 0.763157894736842
------------------------------
Team: Everton
OFF Rating: 1.0526315789473684
DEF Rating: 1.3421052631578947
------------------------------
Team: Sheffield Utd
OFF Rating: 0.9210526315789473
DEF Rating: 2.736842105263158
------------------------------
Team: Brighton
OFF Rating: 1.4473684210526316
DEF Rating: 1.631578947368421
------------------------------
Team: Bournemouth
OFF Rating: 1.4210526315789473
DEF Rating: 1.763157894736842
------------------------------
Team: Newcastle Utd
OFF Rating: 2.236842105263158
DEF Rating: 1.631578947368421
------------------------------
Team: Brentford
OFF Rating: 1.473684210526316
DEF Rating: 1.7105263157894737
------------------------------
Team: Chelsea
OFF Rating: 2.026315789473684
DEF Rating: 1.6578947368421053
------------------------------
Team: Manchester Utd
OFF Rat

In [56]:
# Scrape the next round of fixtures

#start by scraping all fixtures for the season

fixtures = pd.read_html('https://fbref.com/en/comps/9/2023-2024/schedule/2023-2024-Premier-League-Scores-and-Fixtures', # allows us to read tables from html - paste in the link and css selectors
                  attrs={"id":"sched_2023-2024_9_1"})[0] # add the [0] to return the first item in the list - looks better

In [57]:
fixtures.head()

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes
0,1.0,Fri,2023-08-11,20:00,Burnley,0.3,0–3,1.9,Manchester City,21572.0,Turf Moor,Craig Pawson,Match Report,NaN
1,1.0,Sat,2023-08-12,12:30,Arsenal,0.8,2–1,1.2,Nott'ham Forest,59984.0,Emirates Stadium,Michael Oliver,Match Report,NaN
2,1.0,Sat,2023-08-12,15:00,Everton,2.7,0–1,1.5,Fulham,39940.0,Goodison Park,Stuart Attwell,Match Report,NaN
3,1.0,Sat,2023-08-12,15:00,Sheffield Utd,0.5,0–1,1.9,Crystal Palace,31194.0,Bramall Lane,John Brooks,Match Report,NaN
4,1.0,Sat,2023-08-12,15:00,Brighton,4.0,4–1,1.5,Luton Town,31872.0,The American Express Community Stadium,David Coote,Match Report,NaN


In [58]:
#return a certain week of fixtures - we'll use the last gameweek

fixtures_by_gw = fixtures[fixtures['Wk'] == 38.0]

fixtures_by_gw

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes
413,38.0,Sun,2024-05-19,16:00,Luton Town,2.0,2–4,1.1,Fulham,12027.0,Kenilworth Road Stadium,Matt Donohue,Match Report,NaN
414,38.0,Sun,2024-05-19,16:00,Sheffield Utd,1.0,0–3,3.1,Tottenham,29116.0,Bramall Lane,Andy Madley,Match Report,NaN
415,38.0,Sun,2024-05-19,16:00,Manchester City,1.9,3–1,0.4,West Ham,55097.0,Etihad Stadium,John Brooks,Match Report,NaN
416,38.0,Sun,2024-05-19,16:00,Arsenal,2.9,2–1,0.6,Everton,60312.0,Emirates Stadium,Michael Oliver,Match Report,NaN
417,38.0,Sun,2024-05-19,16:00,Brighton,1.7,0–2,1.3,Manchester Utd,31662.0,The American Express Community Stadium,Craig Pawson,Match Report,NaN
418,38.0,Sun,2024-05-19,16:00,Brentford,1.1,2–4,3.4,Newcastle Utd,17124.0,Gtech Community Stadium,Simon Hooper,Match Report,NaN
419,38.0,Sun,2024-05-19,16:00,Chelsea,1.1,2–1,2.2,Bournemouth,39724.0,Stamford Bridge,Anthony Taylor,Match Report,NaN
420,38.0,Sun,2024-05-19,16:00,Crystal Palace,2.5,5–0,0.9,Aston Villa,25191.0,Selhurst Park,Darren Bond,Match Report,NaN
421,38.0,Sun,2024-05-19,16:00,Liverpool,4.5,2–0,0.5,Wolves,60059.0,Anfield,Chris Kavanagh,Match Report,NaN
422,38.0,Sun,2024-05-19,16:00,Burnley,1.2,1–2,1.7,Nott'ham Forest,21109.0,Turf Moor,Graham Scott,Match Report,NaN


In [59]:
# filter for only the home and away teams per fixture

fixtures_nr = fixtures_by_gw[['Home', 'Away']]

fixtures_nr

,Home,Away
413,Luton Town,Fulham
414,Sheffield Utd,Tottenham
415,Manchester City,West Ham
416,Arsenal,Everton
417,Brighton,Manchester Utd
418,Brentford,Newcastle Utd
419,Chelsea,Bournemouth
420,Crystal Palace,Aston Villa
421,Liverpool,Wolves
422,Burnley,Nott'ham Forest


In [99]:
#Now time to run the match simulator for each fixture in GW38

import pandas as pd
import numpy as np
from scipy.stats import poisson
from IPython.display import display



# Define the number of goals for the Poisson matrix
number_of_goals = 7

# Define functions

#create 2 poisson dist for number of goals each team will score
def simulate_poisson_distribution(home_team_xg, away_team_xg):
    score_matrix = np.zeros((number_of_goals, number_of_goals))
    for home_goals in range(number_of_goals):
        for away_goals in range(number_of_goals):
            home_prob = poisson.pmf(home_goals, home_team_xg)
            away_prob = poisson.pmf(away_goals, away_team_xg)
            score_matrix[home_goals][away_goals] = home_prob * away_prob
    return score_matrix

#Display the home and away goals on a heatmap grid
def display_heatmap(result_matrix, home_team, away_team):
    result_df = pd.DataFrame(result_matrix, index=range(number_of_goals), columns=range(number_of_goals))
    
    # Use background_gradient with axis=None to apply gradient to individual cells for easy visual
    styled_df = (
        result_df.style
        .background_gradient(cmap="Oranges", axis=None)
        .set_caption(f'<div style="text-align: center; font-size: 18px;">Score Prediction Heatmap<br>{home_team} vs {away_team}</div>')  # Centered caption with increased font size
        .set_table_attributes('style="text-align: center; width: 800px; height: 400px;"')  # Adjust width and height here
        .format(lambda x: f"{x * 100:.1f}%", na_rep="")  # Format values as percentages
        .set_table_styles(
            [
                {'selector': 'td', 'props': [('text-align', 'center'), ('vertical-align', 'middle'), ('height', '40px')]} ,  # Center text in cells and set height
                {'selector': 'th', 'props': [('text-align', 'center')]}  # Center text in header cells
            ]
        )
    )
    
    # Display the styled DataFrame - for Jupyter Notebook
    display(styled_df)




def calculate_win_draw_probabilities(result_matrix):
    home_win_prob = np.sum(np.tril(result_matrix, -1))
    away_win_prob = np.sum(np.triu(result_matrix, 1))
    draw_prob = np.sum(np.diag(result_matrix))
    return home_win_prob, away_win_prob, draw_prob

# Simulation loop for each fixture
for index, row in fixtures_nr.iterrows():
    home_team = row['Home']
    away_team = row['Away']
    
    # Get xG values
    home_team_xg = team_data.get(home_team, {}).get('OFF Rating', np.nan)
    away_team_xg = team_data.get(away_team, {}).get('DEF Rating', np.nan)
    
    # Check for missing values
    if np.isnan(home_team_xg) or np.isnan(away_team_xg):
        print(f"Missing data for {home_team} vs {away_team}. Skipping this match.")
        continue

    # Run the Poisson simulation
    result_matrix = simulate_poisson_distribution(home_team_xg, away_team_xg)

    # Display the result matrix as a heatmap for the current match
    print(f"\nHeatmap for {home_team} vs {away_team}:")
    display_heatmap(result_matrix, home_team, away_team)
    
    # Calculate 1x2 probabilities
    home_win_prob, away_win_prob, draw_prob = calculate_win_draw_probabilities(result_matrix)
    print(f"\nWin Probabilities for {home_team} vs {away_team}:")
    print(f"Home Win Probability: {home_win_prob:.2f}")
    print(f"Away Win Probability: {away_win_prob:.2f}")
    print(f"Draw Probability: {draw_prob:.2f}")

    # Calculate and print raw prices which is 1 / prob
    home_price = 1 / home_win_prob if home_win_prob > 0 else float('inf')
    draw_price = 1 / draw_prob if draw_prob > 0 else float('inf')
    away_price = 1 / away_win_prob if away_win_prob > 0 else float('inf')

    print("\nRaw Prices:")
    print(f"Home Price : {home_price:.2f}")
    print(f"Draw Price : {draw_price:.2f}")
    print(f"Away Price : {away_price:.2f}")
    print("-" * 40)



Heatmap for Luton Town vs Fulham:


,0,1,2,3,4,5,6
0,5.1%,8.2%,6.6%,3.5%,1.4%,0.5%,0.1%
1,7.0%,11.2%,9.0%,4.8%,1.9%,0.6%,0.2%
2,4.8%,7.7%,6.2%,3.3%,1.3%,0.4%,0.1%
3,2.2%,3.5%,2.8%,1.5%,0.6%,0.2%,0.1%
4,0.7%,1.2%,1.0%,0.5%,0.2%,0.1%,0.0%
5,0.2%,0.3%,0.3%,0.1%,0.1%,0.0%,0.0%
6,0.0%,0.1%,0.1%,0.0%,0.0%,0.0%,0.0%



Win Probabilities for Luton Town vs Fulham:
Home Win Probability: 0.33
Away Win Probability: 0.43
Draw Probability: 0.24

Raw Prices:
Home Price : 3.07
Draw Price : 4.13
Away Price : 2.33
----------------------------------------

Heatmap for Sheffield Utd vs Tottenham:


,0,1,2,3,4,5,6
0,8.0%,12.8%,10.3%,5.5%,2.2%,0.7%,0.2%
1,7.4%,11.8%,9.5%,5.1%,2.0%,0.7%,0.2%
2,3.4%,5.4%,4.4%,2.3%,0.9%,0.3%,0.1%
3,1.0%,1.7%,1.3%,0.7%,0.3%,0.1%,0.0%
4,0.2%,0.4%,0.3%,0.2%,0.1%,0.0%,0.0%
5,0.0%,0.1%,0.1%,0.0%,0.0%,0.0%,0.0%
6,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%



Win Probabilities for Sheffield Utd vs Tottenham:
Home Win Probability: 0.22
Away Win Probability: 0.53
Draw Probability: 0.25

Raw Prices:
Home Price : 4.63
Draw Price : 4.00
Away Price : 1.88
----------------------------------------

Heatmap for Manchester City vs West Ham:


...


Win Probabilities for Crystal Palace vs Aston Villa:
Home Win Probability: 0.36
Away Win Probability: 0.40
Draw Probability: 0.24

Raw Prices:
Home Price : 2.80
Draw Price : 4.20
Away Price : 2.48
----------------------------------------

Heatmap for Liverpool vs Wolves:


,0,1,2,3,4,5,6
0,1.9%,3.2%,2.8%,1.6%,0.7%,0.2%,0.1%
1,4.3%,7.3%,6.2%,3.5%,1.5%,0.5%,0.1%
2,4.8%,8.2%,7.0%,4.0%,1.7%,0.6%,0.2%
3,3.6%,6.2%,5.3%,3.0%,1.3%,0.4%,0.1%
4,2.1%,3.5%,3.0%,1.7%,0.7%,0.3%,0.1%
5,0.9%,1.6%,1.4%,0.8%,0.3%,0.1%,0.0%
6,0.4%,0.6%,0.5%,0.3%,0.1%,0.0%,0.0%



Win Probabilities for Liverpool vs Wolves:
Home Win Probability: 0.50
Away Win Probability: 0.29
Draw Probability: 0.20

Raw Prices:
Home Price : 2.01
Draw Price : 4.98
Away Price : 3.43
----------------------------------------

Heatmap for Burnley vs Nott'ham Forest:


,0,1,2,3,4,5,6
0,5.8%,10.3%,9.1%,5.3%,2.3%,0.8%,0.2%
1,6.3%,11.1%,9.8%,5.7%,2.5%,0.9%,0.3%
2,3.4%,6.0%,5.3%,3.1%,1.4%,0.5%,0.1%
3,1.2%,2.2%,1.9%,1.1%,0.5%,0.2%,0.1%
4,0.3%,0.6%,0.5%,0.3%,0.1%,0.0%,0.0%
5,0.1%,0.1%,0.1%,0.1%,0.0%,0.0%,0.0%
6,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%



Win Probabilities for Burnley vs Nott'ham Forest:
Home Win Probability: 0.23
Away Win Probability: 0.53
Draw Probability: 0.23

Raw Prices:
Home Price : 4.32
Draw Price : 4.26
Away Price : 1.88
----------------------------------------


In [3]:
#Calculating win prob & supremacy

def calculate_win_probability(man_city_rating, liverpool_rating):
    formula = -(man_city_rating - liverpool_rating) / 400
    win_probability = 1 / (1 + 10**formula)
    return win_probability

def calculate_supremacy(man_city_rating, liverpool_rating):
    supremacy = (man_city_rating - liverpool_rating) * 0.462476 / abs(man_city_rating - liverpool_rating)
    return supremacy

# SPI ratings
man_city_rating = 2047
liverpool_rating = 1957

# Calculate and print win probability and supremacy
win_probability = calculate_win_probability(man_city_rating, liverpool_rating)
supremacy = calculate_supremacy(man_city_rating, liverpool_rating)

price = 1 / win_probability  # Calculate Price

print(f"Win Probability for Manchester City: {win_probability:.2f}")
print(f"Supremacy for Manchester City: {supremacy:.2f}")
print(f"Price for Manchester City: {price:.2f}")

Win Probability for Manchester City: 0.63
Supremacy for Manchester City: 0.46
Price for Manchester City: 1.60


In [ ]:
Win probabilities
Home win probability = []
Draw Probability = []
Away win probability = []

Prices
Home price = []
Draw Price = []
Away Price = []

Supremacy = []

Results
Home Goals = []
Away Goals = []

XG
Home Team XG = []
Away Team XG = []


In [1]:
import numpy as np

# Number of dice rolls
num_rolls = 10000

# Simulate rolling two dice
dice_rolls = np.random.randint(1, 7, size=(num_rolls, 2))

# Calculate the sum of each pair of dice
dice_sums = np.sum(dice_rolls, axis=1)

# Count occurrences of each sum
sum_counts = np.bincount(dice_sums, minlength=13)[2:]

# Calculate the percentage of each sum
sum_percentages = (sum_counts / num_rolls) * 100

# Print the count and percentage of each sum
for number, (count, percentage) in enumerate(zip(sum_counts, sum_percentages), start=2):
    print(f'Sum {number}: {count} times ({percentage:.2f}%)')


Sum 2: 250 times (2.50%)
Sum 3: 585 times (5.85%)
Sum 4: 880 times (8.80%)
Sum 5: 1106 times (11.06%)
Sum 6: 1372 times (13.72%)
Sum 7: 1694 times (16.94%)
Sum 8: 1356 times (13.56%)
Sum 9: 1111 times (11.11%)
Sum 10: 810 times (8.10%)
Sum 11: 565 times (5.65%)
Sum 12: 271 times (2.71%)
